In [5]:
import re

%matplotlib inline
import matplotlib

datapath = "data/neko.txt.cabocha"

## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，各文をMorphオブジェクトのリストとして表現し，3文目の形態素列を表示せよ．

### 準備
 ```
 brew install cabocha
 cabocha -f1 neko.txt > neko.txt.cabocha
 ```
 
### memo
クラスに`__str__`を定義するとprint等で呼ばれた時の表示方法を指定できる


In [21]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
        self.jp2en = {"表層形": surface, "基本形": base, "品詞": pos, "品詞細分類1": pos1}
    
    def __str__(self):
        return self.surface
    
    def check(self, speech, kind):
        return self.jp2en[speech] == kind

In [11]:
prg_morph = re.compile(r"(?P<sur>.+?)\t(?P<pos>[^,]+),(?P<pos1>[^,]+),([^,]+,){4}(?P<base>[^,]+).*")

def gen_sentence_morph():
    morph_list = []
    with open(datapath, encoding="utf-8") as f:
        for line in f:
            res_m = prg_morph.match(line)
            if res_m:
                morph_list.append(Morph(res_m.group("sur"), res_m.group("base"), res_m.group("pos"), res_m.group("pos1")))
            elif line == "EOS\n":
                yield morph_list
                morph_list = []

for i, sentence in enumerate(gen_sentence_morph()):
    if i== 3:
        print(list(map(str,sentence)))
        break
            

['名前', 'は', 'まだ', '無い', '。']


## 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，8文目の文節の文字列と係り先を表示せよ．第5章の残りの問題では，ここで作ったプログラムを活用せよ．

### memo
文節、形態素、文末の判定は正規表現でやる

1. 文節

    - 形態素は以降の行に書いてあるので後でappendするための受け皿をつくっておく。
    - 係り元を覚えておくための辞書を作る。日本語は係り先の方が文の後ろにあるから先読みせず1行ずつ処理可能
    - EOSが2行以上連なってるところが1文にカウントされてしまっている
    
2. 形態素

    4章を丸々流用

In [24]:
class Chunk:
    def __init__(self, index, dst, srcs):
        self.morphs = []
        self.index = index
        self.dst = dst
        self.srcs = srcs
    
    def __str__(self):
        "形態素をくっつけたものを表示(句読点除く)"
        return re.sub('[、。「」]', '', ''.join(map(str, self.morphs)))
    
    def __int__(self):
        "文中での通し番号を返す"
        return self.index
    
    def append(self, morph):
        "文節に形態素を追加"
        self.morphs.append(morph)
        
    def get(self, speech, kind):
        "品詞(speech)が特定の種類(kind)の形態素のみを取り出す"
        return [morph for morph in self.morphs if morph.check(speech, kind)]

In [10]:
from collections import defaultdict

prg_morph = re.compile(r"(?P<sur>.+?)\t(?P<pos>[^,]+),(?P<pos1>[^,]+),([^,]+,){4}(?P<base>[^,]+).*")
prg_chunk = re.compile(r"\* (?P<num>\d+) (?P<dst>-?\d+)D \d+/\d+ .*")

def gen_sentence():
    chunk = None # 名前確保用
    sentence, chunk_srcs = [], defaultdict(list)
    with open(datapath, encoding="utf-8") as f:
        for line in f:
            res_m = prg_morph.match(line)
            res_c = prg_chunk.match(line)
            if res_c:
                me, dst = int(res_c.group("num")), int(res_c.group("dst"))
                chunk_srcs[dst].append(me)
                chunk = Chunk(me, dst, chunk_srcs[me])
                sentence.append(chunk)
            elif res_m:
                chunk.append(Morph(res_m.group("sur"), res_m.group("base"), res_m.group("pos"), res_m.group("pos1")))
            else: # End of sentence
                yield sentence
                sentence, chunk_srcs = [], defaultdict(list)

def dependency_parsing(n):
    counter = 0
    for sentence in gen_sentence():
        if sentence:
            counter += 1
            if counter == n:
                print("sentence",counter)
                for chunk in sentence:
                    print(chunk, "\t->", sentence[chunk.dst] if chunk.dst != -1 else None)

dependency_parsing(8)
            

sentence 8
この 	-> 書生というのは
書生というのは 	-> 話である
時々 	-> 捕えて
我々を 	-> 捕えて
捕えて 	-> 煮て
煮て 	-> 食うという
食うという 	-> 話である
話である 	-> None


## 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [67]:
with open("out/42.txt", "w", encoding="utf-8") as f:
    for sentence in gen_sentence():
        for chunk in sentence:
            if chunk.dst == -1:
                continue
                
            line = '\t'.join(map(str, [chunk, sentence[chunk.dst]]))
            f.write(line + "\n")

## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．



In [ ]:
with open("out/43.txt", "w", encoding="utf-8") as f:
    for sentence in gen_sentence():
        for chunk in sentence:
            if chunk.get("品詞", "名詞") and sentence[chunk.dst].get("品詞", "動詞"):
                line = '\t'.join(map(str, [chunk, sentence[chunk.dst]]))
                f.write(line + "\n") if chunk.dst != -1 else None

## 44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

## 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

- 動詞を含む文節において，最左の動詞の基本形を述語とする
- 述語に係る助詞を格とする
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる

### memo
1. 動詞入りの文節を探す
1. 最左動詞を持ってくる
1. 係り元の文節を列挙
1. 係り元の文節内の最右助詞を抽出、ソート

参考: 助詞[(wikipedia)](https://ja.wikipedia.org/wiki/助詞#格助詞)

In [20]:
with open("out/45_cp.txt", "w", encoding="utf-8") as f:
    for sentence in gen_sentence():
        for chunk in sentence:
            verbs_in_chunk = chunk.get("品詞", "動詞")
            if verbs_in_chunk:
                predicate, *rest = verbs_in_chunk # 最左動詞
                chunk_srcs = [sentence[i] for i in chunk.srcs]
                particles_in_chunks = [chunk_src.get("品詞", "助詞") for chunk_src in chunk_srcs]
                particles = [ps[-1].base for ps in particles_in_chunks if ps]
                if particles:
                    f.write(''.join([predicate.base, '\t', ' '.join(sorted(particles)), '\n']))


# 動詞→助詞のペアを列挙
with open("out/45_cp_all.txt", "w", encoding="utf-8") as f:
    for sentence in gen_sentence():
        for chunk in sentence:
            verbs_in_chunk = chunk.get("品詞", "動詞")
            if verbs_in_chunk:
                predicate, *rest = verbs_in_chunk # 最左動詞
                chunk_srcs = [sentence[i] for i in chunk.srcs]
                particles_in_chunks = [chunk_src.get("品詞", "助詞") for chunk_src in chunk_srcs]
                particles = [ps[-1].base for ps in particles_in_chunks if ps]
                for p in particles:
                    f.write(''.join([predicate.base, '\t', p, '\n']))

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

- コーパス中で頻出する述語と格パターンの組み合わせ

```sort out/45_cp_all.txt | sort -f | uniq -c | sort -k 1r,1 -k 2f,2 > out/45.count```

- 「する」「見る」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

```grep -E "^する" out/45_cp_all.txt | cut -f 2- | sort | uniq -c | sort -rk 1 | awk '{print $2}' | tr '\n' ' ' > out/45.suru```  
```grep -E "^見る" out/45_cp_all.txt | cut -f 2- | sort | uniq -c | sort -rk 1 | awk '{print $2}' | tr '\n' ' ' > out/45.miru```  
```grep -E "^与える" out/45_cp_all.txt | cut -f 2- | sort | uniq -c | sort -rk 1 | awk '{print $2}' | tr '\n' ' ' > out/45.ataeru```  

## 46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

- 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
- 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる

### memo
45に加えて...  
タプルで持って助詞で辞書順ソートした後に各々を分けて列挙

In [ ]:
from itertools import chain

with open("out/46.txt", "w", encoding="utf-8") as f:
    for sentence in gen_sentence():
        for chunk in sentence:
            verbs_in_chunk = chunk.get("品詞", "動詞")
            if verbs_in_chunk:
                predicate, *rest = verbs_in_chunk
                chunk_srcs = [sentence[i] for i in chunk.srcs]
                particles_in_chunks = [chunk.get("品詞", "助詞") for chunk_src in chunk_srcs]
                pscs = [(str(ps[-1]), str(cs)) for ps, cs in zip(particles_in_chunks, chunk_srcs) if ps]
                pscs.sort(key=lambda t: t[0])
                ps, cs =[p for p, c in pscs], [c for p, c in pscs]
                if pscs:
                    f.write(''.join([predicate.base, '\t', ' '.join(ps), '\t', ' '.join(cs), '\n']))

## 47. 機能動詞構文のマイニング
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

- 「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
- 述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
- 述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）

### memo
46に加えて...  
- 文節が「サ変接続→を」を持つかどうか調べる
- 持ってたらそいつの係り先が動詞を持つか探す、あとは46と同様
- 出力フォーマットを少し修正する

In [ ]:
chunk2sw = lambda chunk: ''.join(["s" if morph.pos1 == "サ変接続" else "w" if str(morph) == "を" else "x" for morph in chunk.morphs])
chunk2xp = lambda chunk: ''.join(["p" if morph.pos == "助詞" else "x" for morph in chunk.morphs])
prg_find_sw = re.compile("sw")
prg_last_particle = re.compile("p[^p]*?$")

with open("out/47.txt", "w", encoding="utf-8") as f:
    for sentence in gen_sentence():
        for chunk in sentence:
            # 「サ変->を」の抽出
            match_sw = prg_find_sw.search(chunk2sw(chunk))        
            if match_sw:
                sahen = chunk.morphs[match_sw.start()].surface # サ変名詞を取得
                verbs_in_chunk = sentence[chunk.dst].get("品詞","動詞")
                if verbs_in_chunk:
                    predicate, *rest = verbs_in_chunk # 最左動詞を取得
                    chunk_srcs = [sentence[i] for i in sentence[chunk.dst].srcs if not i == chunk.index] # 係り元の列挙
                    matches_p = [prg_last_particle.search(chunk2xp(chunk_src)) for chunk_src in chunk_srcs] # 最右助詞の探索
                    pscs = [(str(cs.morphs[m.start()]), str(cs)) for m, cs in zip(matches_p, chunk_srcs) if m is not None] # タプルの作成と文字列化
                    if pscs:
                        pscs.sort(key=lambda t: t[0])
                        ps, cs =[p for p, c in pscs], [c for p, c in pscs]
                        f.write(''.join([sahen, 'を', predicate.base, '\t', ' '.join(ps), '\t', ' '.join(cs), '\n']))

## 48. 名詞から根へのパスの抽出
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

- 各文節は（表層形の）形態素列で表現する
- パスの開始文節から終了文節に至るまで，各文節の表現を"->"で連結する

In [29]:
with open("out/48.txt", "w", encoding="utf-8") as f:
    for sentence in gen_sentence():
        for chunk in sentence:
            if chunk.get("品詞", "名詞"):
                now_chunk, path = chunk, []
                while True:
                    path.append(str(now_chunk))
                    if now_chunk.dst == -1:
                        break
                    
                    now_chunk = sentence[now_chunk.dst]
                #print(" -> ".join(path))
                f.write(" -> ".join(path))

一
吾輩は -> 猫である
猫である
名前は -> 無い
　どこで -> 生れたか -> つかぬ
見当が -> つかぬ
何でも -> 薄暗い -> 所で -> 泣いて -> 記憶している
所で -> 泣いて -> 記憶している
ニャーニャー -> 泣いて -> 記憶している
いた事だけは -> 記憶している
記憶している
吾輩は -> 見た
ここで -> 始めて -> 人間という -> ものを -> 見た
人間という -> ものを -> 見た
ものを -> 見た
あとで -> 聞くと -> 種族であったそうだ
それは -> 種族であったそうだ
書生という -> 人間中で -> 種族であったそうだ
人間中で -> 種族であったそうだ
一番 -> 獰悪な -> 種族であったそうだ
獰悪な -> 種族であったそうだ
種族であったそうだ
書生というのは -> 話である
我々を -> 捕えて -> 煮て -> 食うという -> 話である
話である
当時は -> なかったから -> 思わなかった
何という -> 考も -> なかったから -> 思わなかった
考も -> なかったから -> 思わなかった
彼の -> 掌に -> 載せられて -> 持ち上げられた -> 時 -> フワフワした -> 感じが -> あったばかりである
掌に -> 載せられて -> 持ち上げられた -> 時 -> フワフワした -> 感じが -> あったばかりである
スーと -> 持ち上げられた -> 時 -> フワフワした -> 感じが -> あったばかりである
時 -> フワフワした -> 感じが -> あったばかりである
感じが -> あったばかりである
掌の -> 上で -> 落ちついて -> 見たのが -> 人間という -> ものの -> 見始であろう
上で -> 落ちついて -> 見たのが -> 人間という -> ものの -> 見始であろう
書生の -> 顔を -> 見たのが -> 人間という -> ものの -> 見始であろう
顔を -> 見たのが -> 人間という -> ものの -> 見始であろう
見たのが -> 人間という -> ものの -> 見始であろう
人間という -> ものの -> 見始であろう
ものの -> 見始であろう
見始であろう
時 -> ものだと -> 思った -> 感じ

読者に -> 断っておきたいが -> よくない
人間が -> 何ぞと -> いうと -> よくない
猫々と -> 評価する -> 癖が -> あるは -> よくない
事もなげに -> もって -> 評価する -> 癖が -> あるは -> よくない
軽侮の -> 口調を -> もって -> 評価する -> 癖が -> あるは -> よくない
口調を -> もって -> 評価する -> 癖が -> あるは -> よくない
吾輩を -> 評価する -> 癖が -> あるは -> よくない
評価する -> 癖が -> あるは -> よくない
癖が -> あるは -> よくない
人間の -> 糟から -> 出来て -> 考えるのは -> 事でもあろうが -> 者じゃない
糟から -> 出来て -> 考えるのは -> 事でもあろうが -> 者じゃない
牛と -> 馬が -> 出来て -> 考えるのは -> 事でもあろうが -> 者じゃない
馬が -> 出来て -> 考えるのは -> 事でもあろうが -> 者じゃない
牛と -> 馬の -> 糞から -> 製造されたごとく -> 考えるのは -> 事でもあろうが -> 者じゃない
馬の -> 糞から -> 製造されたごとく -> 考えるのは -> 事でもあろうが -> 者じゃない
糞から -> 製造されたごとく -> 考えるのは -> 事でもあろうが -> 者じゃない
猫が -> 製造されたごとく -> 考えるのは -> 事でもあろうが -> 者じゃない
製造されたごとく -> 考えるのは -> 事でもあろうが -> 者じゃない
考えるのは -> 事でもあろうが -> 者じゃない
自分の -> 無智に -> 心付かんで -> する -> 教師などには -> 事でもあろうが -> 者じゃない
無智に -> 心付かんで -> する -> 教師などには -> 事でもあろうが -> 者じゃない
高慢な -> 顔を -> する -> 教師などには -> 事でもあろうが -> 者じゃない
顔を -> する -> 教師などには -> 事でもあろうが -> 者じゃない
教師などには -> 事でもあろうが -> 者じゃない
ありがちの -> 事でもあろうが -> 者じゃない
事でもあろうが -> 者じゃない
はたから -> 見て -> 見っとも 

主人さえ -> 出て来て -> いった
書斎から -> 出て来て -> いった
馬鹿野郎と -> いった
云うのは -> 小供ばかりである
小供ばかりである
みんな -> 申し合せたように -> 笑っている
申し合せたように -> 笑っている
腹は -> 立つ -> 苦しくはある -> 弱った
踊は -> 弱った
訳に -> ゆかぬ -> 弱った
笑いが -> やみそうになったら -> なる -> 女の子が -> いったので -> 笑われた
やみそうになったら -> なる -> 女の子が -> いったので -> 笑われた
五つに -> なる -> 女の子が -> いったので -> 笑われた
女の子が -> いったので -> 笑われた
様 -> 随分ねと -> いったので -> 笑われた
猫も -> 随分ねと -> いったので -> 笑われた
狂瀾を -> するという -> 勢でまた -> 笑われた
既倒に -> するという -> 勢でまた -> 笑われた
勢でまた -> 笑われた
大変 -> 笑われた
人間の -> 同情に -> 乏しい -> 実行も -> 大分見聞したが -> なかった
同情に -> 乏しい -> 実行も -> 大分見聞したが -> なかった
実行も -> 大分見聞したが -> なかった
大分見聞したが -> なかった
時 -> 感じた -> 事は -> なかった
事は -> なかった
天祐も -> 消え失せて -> なって -> 白黒するの -> 醜態を -> 演ずるまでに -> 閉口した
どっかへ -> 消え失せて -> なって -> 白黒するの -> 醜態を -> 演ずるまでに -> 閉口した
在来の -> 通り四つ這に -> なって -> 白黒するの -> 醜態を -> 演ずるまでに -> 閉口した
通り四つ這に -> なって -> 白黒するの -> 醜態を -> 演ずるまでに -> 閉口した
眼を -> 白黒するの -> 醜態を -> 演ずるまでに -> 閉口した
白黒するの -> 醜態を -> 演ずるまでに -> 閉口した
醜態を -> 演ずるまでに -> 閉口した
閉口した
見殺しに -> するのも -> 気の毒と -> 見えて -> 命ずる
するのも -> 気の毒と -> 見えて -> 命ずる
気の毒と -> 見えて -> 命ずる


障子の -> 中で -> しているのを -> 隠れて -> 聞いていると -> こうであった
中で -> しているのを -> 隠れて -> 聞いていると -> こうであった
例の -> 御師匠さんと -> しているのを -> 隠れて -> 聞いていると -> こうであった
御師匠さんと -> しているのを -> 隠れて -> 聞いていると -> こうであった
下女が -> しているのを -> 隠れて -> 聞いていると -> こうであった
しているのを -> 隠れて -> 聞いていると -> こうであった
手水鉢の -> 葉蘭の -> 影に -> 隠れて -> 聞いていると -> こうであった
葉蘭の -> 影に -> 隠れて -> 聞いていると -> こうであった
影に -> 隠れて -> 聞いていると -> こうであった
三毛は -> たべるかい -> 寝かしておきました -> 猫らしくない
御飯を -> たべるかい -> 寝かしておきました -> 猫らしくない
今朝から -> 食べません -> あったかに -> して -> 寝かしておきました -> 猫らしくない
何にも -> 食べません -> あったかに -> して -> 寝かしておきました -> 猫らしくない
御火燵に -> 寝かしておきました -> 猫らしくない
猫らしくない
人間の -> 取扱を -> 受けている
取扱を -> 受けている
自分の -> 境遇と -> 比べて -> 羨ましくもあるが -> 嬉しくもある
境遇と -> 比べて -> 羨ましくもあるが -> 嬉しくもある
己が -> 愛している -> 猫が -> 受けていると -> 思えば -> 嬉しくもある
猫が -> 受けていると -> 思えば -> 嬉しくもある
かくまで -> 受けていると -> 思えば -> 嬉しくもある
厚遇を -> 受けていると -> 思えば -> 嬉しくもある
御飯を -> たべないと -> 疲れるばかりだからね -> もの
身体が -> 疲れるばかりだからね -> もの
私共でさえ -> いただかないと -> もの
一日御※を -> いただかないと -> もの
明くる日は -> もの
もの
　下女は -> する
自分より -> 動物であるような -> 返事を -> する
猫の -> 方が -> 動物であるよう

服薬でもしたら -> 全快するだろうと -> 帰らない
四時前には -> 全快するだろうと -> 帰らない
全快するだろうと -> 帰らない
細君と -> して -> やると -> 帰らない
相談を -> して -> やると -> 帰らない
甘木医学士を -> やると -> 帰らない
迎いに -> やると -> 帰らない
昨夜が -> 帰らない
当番で -> 帰らない
大学から -> 帰らない
二時頃には -> なりますから -> 返事である
御帰りに -> なりますから -> 返事である
帰り次第 -> 上げますと -> 云う -> 返事である
返事である
今杏仁水でも -> 飲めば -> いるんだが -> もので -> 外れそうになって来る
四時前には -> いるんだが -> もので -> 外れそうになって来る
癒るに -> いるんだが -> もので -> 外れそうになって来る
極って -> いるんだが -> もので -> 外れそうになって来る
いるんだが -> もので -> 外れそうになって来る
運の -> 悪い -> 時には -> もので -> 外れそうになって来る
時には -> もので -> 外れそうになって来る
何事も -> 思うように -> 行かん -> もので -> 外れそうになって来る
思うように -> 行かん -> もので -> 外れそうになって来る
もので -> 外れそうになって来る
妻君の -> 喜ぶ -> 笑顔を -> 見て -> 云う -> 予算も -> 外れそうになって来る
笑顔を -> 見て -> 云う -> 予算も -> 外れそうになって来る
楽も -> うと -> 云う -> 予算も -> 外れそうになって来る
予算も -> 外れそうになって来る
外れそうになって来る
細君は -> いらっしゃれませんかと -> 聞く
顔付を -> して -> いらっしゃれませんかと -> 聞く
四時までには -> 見せるから -> 安心しているが -> いい
安心しているが -> いい
顔でも -> 洗って -> 着換えて -> 待っているがいいと -> 感慨である
着物でも -> 着換えて -> 待っているがいいと -> 感慨である
口では -> 感慨である
云ったような -> ものの -> 胸中は -> 感慨である
ものの -> 胸中は ->

...

妙ね
妙ですとも -> 大妙ですわ
大妙ですわ
保険の -> 掛金を -> 出す -> くらいなら -> すまし切っているんですよ
掛金を -> 出す -> くらいなら -> すまし切っているんですよ
銀行へ -> 貯金する -> 方が -> すまし切っているんですよ
貯金する -> 方が -> すまし切っているんですよ
方が -> すまし切っているんですよ
遥かに -> すまし切っているんですよ
ましだって -> すまし切っているんですよ
すまし切っているんですよ
貯金が -> あるの？
もんですか
自分が -> 死んだ -> あとなんか -> 考なんかないんですよ
あとなんか -> 考なんかないんですよ
考なんかないんですよ
心配ね
なんでしょう -> 方だって -> いないわね
ここへ -> いらっしゃる -> 方だって -> いないわね
方だって -> いないわね
叔父さんのようなのは -> いないわね
一人も -> いないわね
ものですか
無類ですよ
鈴木さんにでも -> 頼んで -> 意見でもして -> 貰うといいんですよ
意見でもして -> 貰うといいんですよ
貰うといいんですよ
穏やかな -> 人だと -> 楽ですがねえ
人だと -> 楽ですがねえ
楽ですがねえ
鈴木さんは -> わるいのよ
うちじゃ -> わるいのよ
評判が -> わるいのよ
みんな -> 逆なのね
逆なのね
それじゃ -> 落ちついてる——
方が -> いいでしょう——ほらあの -> 落ちついてる——
いいでしょう——ほらあの -> 落ちついてる——
落ちついてる——
八木さん？
八木さんには -> 大分閉口しているんですがね
大分閉口しているんですがね
昨日 -> 来て -> いった -> ものだから -> 利かないかも知れない
迷亭さんが -> 来て -> いった -> ものだから -> 利かないかも知れない
悪口を -> いった -> ものだから -> 利かないかも知れない
ものだから -> 利かないかも知れない
風に -> 落ちついていれば—— -> なすったわ
鷹揚に -> 落ちついていれば—— -> なすったわ
落ちついていれば—— -> なすったわ
こないだ学校で -> なすったわ
演説を -> なすったわ
八木さんが -> ？
八木さんは -> 先生なの
雪江さん

猫より -> 受けて -> よろしい
尊敬を -> 受けて -> よろしい
時は -> やめてしまうつもりである
吾輩も -> 書くのは -> 気の毒だから -> やめてしまうつもりである
いたずらを -> 書くのは -> 気の毒だから -> やめてしまうつもりである
書くのは -> 気の毒だから -> やめてしまうつもりである
気の毒だから -> やめてしまうつもりである
やめてしまうつもりである
自分で -> 分らないと -> 同じように -> かけるのであろう
自分の -> 鼻の -> 高さが -> 分らないと -> 同じように -> かけるのであろう
鼻の -> 高さが -> 分らないと -> 同じように -> かけるのであろう
高さが -> 分らないと -> 同じように -> かけるのであろう
同じように -> かけるのであろう
自己の -> 何物かは -> かけるのであろう
何物かは -> かけるのであろう
見当が -> つき -> いと見えて -> 軽蔑している -> 猫に -> 向ってさえかような -> 質問を -> かけるのであろう
平生から -> 軽蔑している -> 猫に -> 向ってさえかような -> 質問を -> かけるのであろう
軽蔑している -> 猫に -> 向ってさえかような -> 質問を -> かけるのであろう
猫に -> 向ってさえかような -> 質問を -> かけるのであろう
質問を -> かけるのであろう
かけるのであろう
人間は -> 抜けている
生意気なようでも -> 抜けている
どこか -> 抜けている
万物の -> 霊だなどと -> 担いで -> あるくかと -> 思うと -> 理解出来ない
霊だなどと -> 担いで -> あるくかと -> 思うと -> 理解出来ない
どこへでも -> 担いで -> あるくかと -> 思うと -> 理解出来ない
万物の -> 霊を -> 担いで -> あるくかと -> 思うと -> 理解出来ない
霊を -> 担いで -> あるくかと -> 思うと -> 理解出来ない
これしきの -> 事実が -> 理解出来ない
事実が -> 理解出来ない
理解出来ない
恬として -> 至っては -> 催したくなる
一※を -> 催したくなる
彼は -> 騒ぎ立てている
万物の -> 霊を -> 担いで 

例は -> ありますよと -> 代って -> 取った
寒月君が -> 代って -> 取った
天下の -> 妻君に -> 代って -> 取った
妻君に -> 代って -> 取った
弁護の -> 労を -> 取った
労を -> 取った
僕も -> 賛成する
寒月君に -> 賛成する
賛成する
僕の -> 考では -> あるばかりで -> 恋だ
考では -> あるばかりで -> 恋だ
人間が -> 入るには -> あるばかりで -> 恋だ
絶対の -> 域に -> 入るには -> あるばかりで -> 恋だ
域に -> 入るには -> あるばかりで -> 恋だ
二つの -> 道が -> あるばかりで -> 恋だ
道が -> あるばかりで -> 恋だ
二つの -> 道とは -> 恋だ
道とは -> 恋だ
芸術と -> 恋だ
恋だ
夫婦の -> 愛は -> ものだから -> 訳だと -> 思うんだ
愛は -> ものだから -> 訳だと -> 思うんだ
一つを -> 代表する -> ものだから -> 訳だと -> 思うんだ
代表する -> ものだから -> 訳だと -> 思うんだ
ものだから -> 訳だと -> 思うんだ
人間は -> して -> 訳だと -> 思うんだ
是非結婚を -> して -> 訳だと -> 思うんだ
幸福を -> 完うしなければ -> 背く -> 訳だと -> 思うんだ
完うしなければ -> 背く -> 訳だと -> 思うんだ
天意に -> 背く -> 訳だと -> 思うんだ
訳だと -> 思うんだ
思うんだ
——が -> どうでしょう -> 先生と -> 向き直った
先生と -> 向き直った
東風君は -> 向き直った
真面目で -> 向き直った
迷亭君の -> 方へ -> 向き直った
方へ -> 向き直った
御名論だ
僕などは -> 這入れそうもない
絶対の -> 境に -> 這入れそうもない
境に -> 這入れそうもない
這入れそうもない
妻を -> 貰えば -> 這入れやしないと -> 云った
主人は -> 云った
顔を -> して -> 云った
我々未婚の -> 青年は -> 分からないですから -> きいているのです
青年は -> 分からないですから -> きいているのです
芸術の -> 霊気に -> ふれて -> 開拓しなければ -> 分から

声が -> した
声が -> 起して -> 渡ったと -> 思ったら -> 帰った……
反響を -> 起して -> 渡ったと -> 思ったら -> 帰った……
満山の -> 秋の -> 梢を -> 渡ったと -> 思ったら -> 帰った……
秋の -> 梢を -> 渡ったと -> 思ったら -> 帰った……
梢を -> 渡ったと -> 思ったら -> 帰った……
野分と共に -> 渡ったと -> 思ったら -> 帰った……
我に -> 帰った……
安心したと -> 撫でおろす -> 真似を -> する
迷亭君が -> 撫でおろす -> 真似を -> する
胸を -> 撫でおろす -> 真似を -> する
真似を -> する
大死一番乾坤新なりと -> する
独仙君は -> する
目くばせを -> する
寒月君には -> 通じない
我に -> 帰って -> 見廻 -> わすと -> しない
あたりを -> 見廻 -> わすと -> しない
庚申山 -> 一面は -> しない
一面は -> しない
しんとして -> しない
雨垂れほどの -> 音も -> しない
音も -> しない
今の -> 音は -> 考えた
音は -> 考えた
何だろうと -> 考えた
人の -> 声に -> しては -> 鋭すぎるし -> 大き過ぎるし -> しては—— -> おるまい
声に -> しては -> 鋭すぎるし -> 大き過ぎるし -> しては—— -> おるまい
鳥の -> 声に -> しては -> 大き過ぎるし -> しては—— -> おるまい
声に -> しては -> 大き過ぎるし -> しては—— -> おるまい
猿の -> 声に -> しては—— -> おるまい
声に -> しては—— -> おるまい
しては—— -> おるまい
辺に -> おるまい
猿は -> おるまい
何だろう？
何だろうと -> 云う -> 問題が -> 起ると -> 云うので -> かけ廻る
問題が -> 起ると -> 云うので -> かけ廻る
頭の -> なかに -> 起ると -> 云うので -> かけ廻る
なかに -> 起ると -> 云うので -> かけ廻る
これを -> 解釈しようと -> 云うので -> かけ廻る
解釈しようと -> 云うので -> かけ廻る
今まで -> 静まり返っていた 

夫と -> 合わなくなる
自然の -> 勢夫と -> 衝突する
勢夫と -> 衝突する
衝突する
賢妻と -> 名が -> つく -> 以上は -> 衝突している
名が -> つく -> 以上は -> 衝突している
以上は -> 衝突している
朝から -> 晩まで -> 衝突している
晩まで -> 衝突している
夫と -> 衝突している
衝突している
結構な -> 事だが -> 増してくる
事だが -> 増してくる
賢妻を -> 迎えれば -> 迎える -> ほど -> 増してくる
双方共苦しみの -> 程度が -> 増してくる
程度が -> 増してくる
水と -> 油のように -> あって -> 落ちついて -> まだしもだが -> 上がったり下がったりする
油のように -> あって -> 落ちついて -> まだしもだが -> 上がったり下がったりする
夫婦の -> 間には -> あって -> 落ちついて -> まだしもだが -> 上がったり下がったりする
間には -> あって -> 落ちついて -> まだしもだが -> 上がったり下がったりする
截然たるし -> きりが -> あって -> 落ちついて -> まだしもだが -> 上がったり下がったりする
きりが -> あって -> 落ちついて -> まだしもだが -> 上がったり下がったりする
それも -> 落ちついて -> まだしもだが -> 上がったり下がったりする
しきりが -> 保っていれば -> まだしもだが -> 上がったり下がったりする
水平線を -> 保っていれば -> まだしもだが -> 上がったり下がったりする
水と -> 油が -> きかけるのだから -> 上がったり下がったりする
油が -> きかけるのだから -> 上がったり下がったりする
双方から -> きかけるのだから -> 上がったり下がったりする
働ら -> きかけるのだから -> 上がったり下がったりする
きかけるのだから -> 上がったり下がったりする
家の -> なかは -> 上がったり下がったりする
なかは -> 上がったり下がったりする
大地震のように -> 上がったり下がったりする
上がったり下がったりする
ここにおいて -> 分ってくる
夫婦雑居は -> 分ってくる
お互の -> 損だと -> 云う -> 事が ->

## 49. 名詞間の係り受けパスの抽出
文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がi
とj（i<j）のとき，係り受けパスは以下の仕様を満たすものとする．

- 問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を"->"で連結して表現する
- 文節iとjに含まれる名詞句はそれぞれ，XとYに置換する

また，係り受けパスの形状は，以下の2通りが考えられる．

1. 文節iから構文木の根に至る経路上に文節jが存在する場合

文節iから文節jのパスを表示

2. 上記以外で，文節iと文節jから構文木の根に至る経路上で共通の文節kで交わる場合  

文節iから文節kに至る直前のパスと文節jから文節kに至る直前までのパス，文節kの内容を"|"で連結して表示

In [65]:
from itertools import combinations

def chunk_path(sentence, start_index):
    chunk, path = sentence[start_index], []
    while True:
        path.append(chunk)
        if chunk.dst == -1:
            break
            
        chunk = sentence[chunk.dst]
        
    return path

with open("out/49.txt", "w", encoding="utf-8") as f:
    for n, sentence in enumerate(gen_sentence()):
        f.write("sentence "+str(n)+"\n")
        indexes = [chunk.index for chunk in sentence if chunk.get("品詞", "名詞")]
        if len(indexes) < 2:
            continue
            
        for i, j in combinations(indexes, 2):
            path_i = chunk_path(sentence, i)
            path_i_indexes = list(map(int, path_i))
            chunk_i, *rest_i = path_i
            subX = ''.join(["X" if morph.check("品詞","名詞") else str(morph) for morph in chunk_i.morphs])
            
            if j in path_i_indexes:    
                end = list(map(int, rest_i)).index(j)
        
                f.write(' -> '.join([subX] + list(map(str,rest_i[:end])) + ["Y"]) + "\n")
            else:
                path_j = chunk_path(sentence, j)
                path_j_indexes = list(map(int, path_j))
                chunk_j, *rest_j = path_j
                subY = ''.join(["Y" if morph.check("品詞","名詞") else str(morph) for morph in chunk_j.morphs])
                k = min(set(path_i_indexes) & set(path_j_indexes))
                string_k = str(sentence[k])
                
                end_i = list(map(int, rest_i)).index(k)
                string_ik = ' -> '.join([subX] + list(map(str,rest_i[:end_i])))
                
                end_j = list(map(int, rest_j)).index(k)
                string_jk = ' -> '.join([subY] + list(map(str,rest_j[:end_j])))
                
                f.write(" | ".join([string_ik, string_jk, string_k]) + "\n")
                